<a href="https://colab.research.google.com/github/ShubhamSainger/Pet-Classification-Using-CNN/blob/main/Pet_Classification_Model_Using_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importing all essential libraries 
import numpy as np
import pandas as pd
import tensorflow as tf
import re
import zipfile
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#loading zipfile of dataset
zip_file = zipfile.ZipFile(r'/content/Pet Classification.zip')
zip_file.extractall()

# New Section

In [ ]:
df_train = pd.DataFrame(columns=['Class','Location','File'])
df_test = pd.DataFrame(columns=['Class','Location','File'])

In [ ]:
# Creating and arranging train and test data
for i in zip_file.namelist():
  if re.search("train",i):
    img_class = i[i.find('n/')+2 : i.rfind('/')]
    img_loc = i[:i.rfind('/')]
    img_file = i
    df_train.loc[i] = [img_class,img_loc,img_file]
  else:
    img_class = i[i.find('t/')+2 : i.rfind('/')]
    img_loc = i[:i.rfind('/')]
    img_file = i
    df_test.loc[i] = [img_class,img_loc,img_file]

In [ ]:
# Aranging index of datasets
df_train = df_train.set_index(np.arange(0,40))
df_test = df_test.set_index(np.arange(0,20))

In [ ]:
label = df_train.Class.unique().tolist()
img_size = 400
img_depth = 3

In [ ]:
# Batch-gentrator 

def Batch_Genrator(df,batchsize,train_mode=True):
  img_generator= tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=20,
                                                                   width_shift_range=0.2,
                                                                   height_shift_range=0.2,
                                                                   horizontal_flip=True)
  while True:
    batch_nums = np.random.randint(0,df.shape[0],batchsize)
    batch_img = np.zeros((batchsize,img_size,img_size,img_depth))
    batch_label = np.zeros((batchsize,len(label)))

    for i in range(batchsize):
      
      img = tf.keras.preprocessing.image.load_img(df.loc[batch_nums[i]]['File'],target_size=(img_size,img_size)) #loading image
      img = tf.keras.preprocessing.image.img_to_array(img) # convrting it to array
      if(train_mode):
                #Apply transform
                img =  img_generator.random_transform(img)

      img_class = df.loc[batch_nums[i]]['Class']
      img_class = label.index(img_class)
      img_class = tf.keras.utils.to_categorical(img_class,len(label))

      batch_img[i] = img
      batch_label[i] = img_class

    yield batch_img,batch_label

In [ ]:
# Building CNN model

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.BatchNormalization(input_shape = (img_size,img_size,3,)))
model.add(tf.keras.layers.Conv2D(32,kernel_size=(5,5),activation=tf.keras.layers.LeakyReLU(alpha=0.1)))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=2))
model.add(tf.keras.layers.Conv2D(64,kernel_size=(5,5),activation=tf.keras.layers.LeakyReLU(alpha=0.1)))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=2))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(20,activation=tf.keras.layers.LeakyReLU(alpha=0.1)))
model.add(tf.keras.layers.Dropout(0.4))

#Output layer
model.add(tf.keras.layers.Dense(len(label),activation='softmax'))

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.01,)
model.compile(optimizer=opt,loss = 'categorical_crossentropy',metrics='accuracy')

In [ ]:
#train and test genrator
batchsize = 10
train_genrator = Batch_Genrator(df_train,batchsize)
test_genrator = Batch_Genrator(df_test,batchsize,train_mode = False)

In [ ]:
#model training
model.fit(train_genrator,
          epochs = 200,
          steps_per_epoch = df_train.shape[0]//batchsize,
          validation_data = test_genrator,
          validation_steps = df_test.shape[0]//batchsize)

Epoch 1/200
4/4 [==============================] - 1s 366ms/step - loss: 1563.6886 - accuracy: 0.5000 - val_loss: 4473.9580 - val_accuracy: 0.4000
Epoch 2/200
4/4 [==============================] - 2s 384ms/step - loss: 311.9997 - accuracy: 0.5500 - val_loss: 408.0122 - val_accuracy: 0.2500
Epoch 3/200
4/4 [==============================] - 2s 400ms/step - loss: 236.2317 - accuracy: 0.5000 - val_loss: 14963.0283 - val_accuracy: 0.7000
Epoch 4/200
4/4 [==============================] - 1s 374ms/step - loss: 661.4995 - accuracy: 0.6250 - val_loss: 14707.7012 - val_accuracy: 0.5000
Epoch 5/200
4/4 [==============================] - 2s 387ms/step - loss: 819.5734 - accuracy: 0.5000 - val_loss: 2735.3579 - val_accuracy: 0.4500
Epoch 6/200
4/4 [==============================] - 2s 389ms/step - loss: 513.6273 - accuracy: 0.4750 - val_loss: 104.9931 - val_accuracy: 0.6000
Epoch 7/200
4/4 [==============================] - 1s 375ms/step - loss: 242.3638 - accuracy: 0.5250 - val_loss: 2218.9768 

In [ ]:
This Model got 95% validation accuracy. loss and other factor could be better if the train data was more.
We can use transfer learning too to increase its performance.